!pip install git+https://github.com/CC-RMD-EpiBio/autoencirt.git

In [3]:
import numpy as np

In [5]:
import autoencirt
from autoencirt.irt import GRModel
from autoencirt.data.rwa import item_text, get_data

In [6]:
import tensorflow as tf

In [8]:
data, num_people = get_data(reorient=True)
item_names = [f"Q{j}" for j in range(1, 23)]
grm = GRModel(
    data=data,
    item_keys=item_names,
    num_people=num_people,
    dim=3,
    xi_scale=1e-2,
    eta_scale=1e-1,
    kappa_scale=1e-1,
    weight_exponent=1,
    response_cardinality=10
)
ds = next(iter(data.batch(121)))
p = grm.surrogate_distribution.sample(13)
grm.log_likelihood(**p, responses=ds)
grm.unormalized_log_prob(**p, data=ds)

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([-53691.05925421, -53827.05020257, -53774.83789856, -53433.51825225,
       -53481.36674761, -53453.67849794, -53490.83676653, -53669.40667065,
       -53586.8227933 , -53599.54033082, -53716.75874963, -53800.52287867,
       -53795.5294934 ])>

In [ ]:
    losses = grm.calibrate_advi(
        num_epochs=50, rel_tol=1e-4, learning_rate=.01)
    
    print(
        grm.calibrated_expectations['discriminations'][0, ..., 0]
        )
    
    losses = grm.calibrate_advi(
        num_epochs=50, rel_tol=1e-4, learning_rate=.01)
    
    print(
        grm.calibrated_expectations['discriminations'][0, ..., 0]
        )
    
    losses = grm.calibrate_advi(
        num_epochs=50, rel_tol=1e-4, learning_rate=.005)
    
    print(
        grm.calibrated_expectations['discriminations'][0, ..., 0]
        )